In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing
from xgboost.sklearn import XGBClassifier
import joblib

In [11]:
df = pd.read_csv('/content/drive/My Drive/PI/Dataset_for_VK_project.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [12]:
df.head()


,uid,has_photo,sex,has_mobile,followers_count,contacts,relatives,relation,personal,activities,...,deg_centr,average_neighbor_degree,average_degree_connectivity,k_nearest_neighbors,degree_centrality,closeness_centrality,betweenness_centrality,diameter,average_shortest_path_length,target
0,648249344,0,2,1.0,0.0,0,1,1,1,1,...,1.000000,6.482493e+08,0.000000,0.000000,1.000000,0.000000,0.000000,0,0,0
1,645333001,1,2,1.0,2.0,0,0,0,0,0,...,0.257143,2.612393e+08,7.508862,7.508862,0.257143,0.588401,0.057143,2,1,0
2,651165709,1,2,1.0,9.0,0,1,1,1,1,...,0.038501,4.023961e+08,34.912425,34.912425,0.038501,0.511661,0.006917,2,1,0
3,649232401,1,1,1.0,0.0,0,0,0,0,0,...,1.000000,6.492324e+08,0.000000,0.000000,1.000000,0.000000,0.000000,0,0,0
4,639664147,0,1,1.0,1.0,0,1,1,1,1,...,0.260000,3.456780e+08,10.625207,10.625207,0.260000,0.584422,0.032174,2,1,0


In [13]:
df.shape

(11766, 40)

In [14]:
df.columns

Index(['uid', 'has_photo', 'sex', 'has_mobile', 'followers_count', 'contacts',
       'relatives', 'relation', 'personal', 'activities', 'music', 'movies',
       'tv', 'books', 'about', 'quotes', 'albums', 'audios', 'followers',
       'friends', 'pages', 'photos', 'subscriptions', 'videos',
       'clips_followers', 'age', 'city', 'country', 'avg_cl', 'trans',
       'deg_centr', 'average_neighbor_degree', 'average_degree_connectivity',
       'k_nearest_neighbors', 'degree_centrality', 'closeness_centrality',
       'betweenness_centrality', 'diameter', 'average_shortest_path_length',
       'target'],
      dtype='object')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11766 entries, 0 to 11765
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   uid                           11766 non-null  int64  
 1   has_photo                     11766 non-null  int64  
 2   sex                           11766 non-null  int64  
 3   has_mobile                    11766 non-null  float64
 4   followers_count               11766 non-null  float64
 5   contacts                      11766 non-null  int64  
 6   relatives                     11766 non-null  int64  
 7   relation                      11766 non-null  int64  
 8   personal                      11766 non-null  int64  
 9   activities                    11766 non-null  int64  
 10  music                         11766 non-null  int64  
 11  movies                        11766 non-null  int64  
 12  tv                            11766 non-null  int64  
 13  b

In [16]:
df['contacts'].value_counts()

,count
contacts,
0,11766
